In [ ]:
import tensorflow as tf
from tensorflow import keras as kr

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from PIL import Image
import struct

tf.__version__,tf.keras.__version__

In [ ]:
IMAGE_TR_FILE_NAME='C:/Mic/projects/AD/machinelearning/dataset/train-images.idx3-ubyte'
LABEL_TR_FILE_NAME='C:/Mic/projects/AD/machinelearning/dataset/train-labels.idx1-ubyte'

IMAGE_TS_FILE_NAME='C:/Mic/projects/AD/machinelearning/dataset/t10k-images.idx3-ubyte'
LABEL_TS_FILE_NAME='C:/Mic/projects/AD/machinelearning/dataset/t10k-labels.idx1-ubyte'

def load_images(file_name):
    binfile=open(file_name,'rb')
    buffers=binfile.read()
    magic, num, rows, cols = struct.unpack_from('>IIII', buffers, 0)
    bits = num * rows * cols
    images = struct.unpack_from('>' + str(bits) + 'B', buffers, struct.calcsize('>IIII'))
    binfile.close()
    images = np.reshape(images, [num, rows * cols])
    return images

def load_labels(file_name):
    binfile = open(file_name, 'rb')
    buffers = binfile.read()
    magic,num = struct.unpack_from('>II', buffers, 0) 
    labels = struct.unpack_from('>' + str(num) + "B", buffers, struct.calcsize('>II'))
    binfile.close()
    labels = np.reshape(labels, [num])
    return labels  

def showImage(buffer,index=1):
    p=plt.subplot(10,10,index)
    p.imshow(buffer, cmap = matplotlib.cm.binary, interpolation="nearest")
    p.axis("off")
    plt.ion()
    plt.show()
    plt.pause(1)

In [ ]:
train_x=load_images(IMAGE_TR_FILE_NAME).reshape((60000,28,28))
train_y=load_labels(LABEL_TR_FILE_NAME)

test_x=load_images(IMAGE_TS_FILE_NAME).reshape((10000,28,28))
test_y=load_labels(LABEL_TS_FILE_NAME)

train_x.shape,train_y.shape,test_x.shape,test_y.shape

In [ ]:
X_train, X_test=tf.cast(train_x/255.0, tf.float32), tf.cast(test_x/255.0, tf.float32)
Y_train, Y_test=tf.cast(train_y,tf.int16), tf.cast(test_y, tf.int16)
type(X_train)

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
#model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='sigmoid'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=64, epochs=10, validation_split=0.2)

In [ ]:
model.evaluate(X_test, Y_test, verbose=2)

In [ ]:
model.save('./model/my_mnist_linear',overwrite=True, save_format='tf',include_optimizer=True)
#model.save_weights('./model/my_mnist_linear', overwrite=True, save_format='tf')

In [ ]:
pred=np.argmax(model.predict(X_test), axis=1)
print(pred)

for i in range(16):
    n=np.random.randint(1,10000)
    plt.subplot(4,4,i+1)
    plt.axis("off")
    plt.imshow(test_x[n], cmap='gray')
    if(test_y[n]==pred[n]):
        res="OK"
    else:
        res="NG"
    plt.title("[%d]-p<%d> %s" %(test_y[n], pred[n],res))
plt.show()


In [ ]:
lmodel=kr.models.load_model('./model/my_mnist_linear')
#model.load_weights('./model/my_mnist_linear')
lmodel.summary()

